# Building your Deep Neural Network: Step by Step

- In this notebook, we will implement all the functions required to build a deep neural network.
- These functions will be imported into 'Deep Neural Network - Application.ipynb' through 
dnn_app_utils.py

## 1 - Packages

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from building_dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2 - Outline

- Initialize the parameters for a two-layer network and for an $L$-layer neural network.
- Implement the forward propagation module (shown in purple in the figure below).
     - Complete the LINEAR part of a layer's forward propagation step (resulting in $z^{[l]}$).
     - ACTIVATION function (relu/sigmoid) is given in utils.
     - Combine the previous two steps into a new [LINEAR->ACTIVATION] forward function.
     - Stack the [LINEAR->RELU] forward function L-1 time (for layers 1 through L-1) and add a [LINEAR->SIGMOID] at the end (for the final layer $L$). This gives us a new L_model_forward function.
- Compute the loss.
- Implement the backward propagation module (denoted in red in the figure below).
    - Complete the LINEAR part of a layer's backward propagation step.
    - Gradient of the ACTIVATE function (relu_backward/sigmoid_backward) is given in utils.
    - Combine the previous two steps into a new [LINEAR->ACTIVATION] backward function.
    - Stack [LINEAR->RELU] backward L-1 times and add [LINEAR->SIGMOID] backward in a new L_model_backward function
- Finally update the parameters.

<img src="images/final outline.png" style="width:800px;height:500px;">
<caption><center> **Figure 1**</center></caption><br>


**Note** that for every forward function, there is a corresponding backward function. That is why at every step of your forward module we will be storing some values in a cache. The cached values are useful for computing gradients. In the backpropagation module we will then use the cache to calculate the gradients. 

## 3 - Initialization

### 3.1 - 2-layer Neural Network

**Outline**:
- The model's structure is: *LINEAR -> RELU -> LINEAR -> SIGMOID*. 
- We use random initialization for the weight matrices `np.random.randn(shape).T * 0.01` with the correct shape.
- We use zero initialization for the biases `np.zeros(shape)`.

In [2]:
def initialize_parameters(n_0, n_1, n_2):
    """
    Argument:
    n_0 -- size of the input layer
    n_1 -- size of the hidden layer
    n_2 -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    w1 -- weight matrix of shape (n_0, n_1)
                    b1 -- bias vector of shape (n_1, 1)
                    w2 -- weight matrix of shape (n_1, n_2)
                    b2 -- bias vector of shape (n_2, 1)
    """
    
    np.random.seed(1) # set up a seed so that output matches everytime
    
    w1 = np.random.randn(n_1, n_0).T * 0.01
    b1 = np.zeros(shape=(n_1, 1))
    w2 = np.random.randn(n_2, n_1).T * 0.01
    b2 = np.zeros(shape=(n_2, 1))
    
    assert (w1.shape == (n_0, n_1))
    assert (b1.shape == (n_1, 1))
    assert (w2.shape == (n_1, n_2))
    assert (b2.shape == (n_2, 1))
    
    parameters = {"w1": w1,
                  "b1": b1,
                  "w2": w2,
                  "b2": b2}
    
    return parameters

### 3.2 - L-layer Neural Network

<table style="width:100%">
    <tr>
        <td>  </td> 
        <td> **Shape of W** </td> 
        <td> **Shape of b**  </td> 
        <td> **Activation** </td>
        <td> **Shape of Activation** </td> 
    <tr>
       <tr>
        <td> **Layer 1** </td> 
        <td> $(12288, n^{[1]})$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $z^{[1]} = {w^{[1]}}^T  X + b^{[1]} $ </td> 
        <td> $(n^{[1]},209)$ </td> 
    <tr>
       <tr>
        <td> **Layer 2** </td> 
        <td> $(n^{[1]}, n^{[2]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$z^{[2]} = {w^{[2]}}^T a^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr> 
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>    
   <tr>
        <td> **Layer L-1** </td> 
        <td> $(n^{[L-2]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$z^{[L-1]} =  {w^{[L-1]}}^T a^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
    <tr>   
   <tr>
        <td> **Layer L** </td> 
        <td> $(n^{[L-1]}, n^{[L]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $z^{[L]} =  {w^{[L]}}^T a^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>
</table>

In [3]:
# initialize_parameters_deep
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "w1", "b1", ..., "wL", "bL":
                    wl -- weight matrix of shape (layer_dims[l-1], layer_dims[l])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L): 
        parameters['w' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]).T * 0.01
        parameters['b' + str(l)] = np.zeros(shape=(layer_dims[l], 1))
        
        assert(parameters['w' + str(l)].shape == (layer_dims[l-1], layer_dims[l]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

## 4 - Forward propagation module

### 4.1 - Linear Forward 

The linear forward module (vectorized over all the examples) computes the following equations:

$$z^{[l]} = {w^{[l]}}^T a^{[l-1]} +b^{[l]}\tag{1}$$

where $a^{[0]} = X$. 

In [4]:
# linear_forward
def linear_forward(a , w, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    a -- activations from previous layer (or input data): (size of previous layer, number of examples)
    w -- weights matrix: numpy array of shape (size of previous layer, size of current layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python dictionary containing "a", "w" and "b" ; stored for computing the backward pass efficiently
    """
    
    z = np.dot(w.T, a) + b
    
    cache = (a, w, b)
    
    return z, cache

### 4.2 - Linear-Activation Forward

We use two activation functions from utils:

- **Sigmoid**: $\sigma(z) = \sigma(w^T a + b) = \frac{1}{ 1 + e^{-(w^T a + b)}}$. This function returns **two** items: the activation value "`a`" and a "`cache`" that contains "`z`" (it's what we will feed in to the corresponding backward function). To use it we can call: 
``` python
a, activation_cache = sigmoid(z)
```

- **ReLU**: The mathematical formula for ReLu is $a = RELU(z) = max(0, z)$. We use the `relu` function from utils. This function returns **two** items: the activation value "`a`" and a "`cache`" that contains "`z`" (it's what we will feed in to the corresponding backward function). To use it we can call:
``` python
a, activation_cache = relu(z)
```

In [5]:
# linear_activation_forward
def linear_activation_forward(a_prev, w, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    a_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    w -- weights matrix: numpy array of shape ( size of previous layer, size of current layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    a -- the output of the activation function, also called the post-activation value 
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        z, linear_cache = linear_forward(a_prev, w, b)
        a, activation_cache = sigmoid(z)
    
    elif activation == "relu":
        z, linear_cache = linear_forward(a_prev, w, b)
        a, activation_cache = relu(z)

    
    assert (a.shape == (w.shape[1], a_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return a, cache

### d) L-Layer Model 

For even more convenience when implementing the $L$-layer Neural Net, we will need a function that replicates the previous one (`linear_activation_forward` with RELU) $L-1$ times, then follows that with one `linear_activation_forward` with SIGMOID.

<img src="images/model_architecture_kiank.png" style="width:600px;height:300px;">
<caption><center> **Figure 2** : *[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* model</center></caption><br>

In [6]:
# L_model_forward
def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    aL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """

    caches = []
    a = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        a_prev = a 
        a, cache = linear_activation_forward(a_prev, parameters["w" + str(l)], parameters["b" + str(l)], activation = "relu")
        caches.append(cache)
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    aL, cache = linear_activation_forward(a, parameters["w" + str(L)], parameters["b" + str(L)], activation = "sigmoid")
    caches.append(cache)
    
    assert(aL.shape == (1, X.shape[1]))
            
    return aL, caches

## 5 - Cost function

The cross-entropy cost $J$, is given by: $$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{2}$$


In [7]:
# compute_cost
def compute_cost(aL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    aL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]
    if Y.shape[1] > 1:
        Y = Y.T # keeping vectors as column vectors

    # Compute loss from aL and y
    cost = -(1/m) * ( np.dot(Y.T, np.log(aL.T)) + np.dot((1-Y).T, np.log(1-aL.T)) )[0, 0]
    
    assert(cost.shape == ())
    
    return cost

## 6 - Backward propagation module

<img src="images/backprop_kiank.png" style="width:650px;height:250px;">
<caption><center> **Figure 3** : Forward and Backward propagation for *LINEAR->RELU->LINEAR->SIGMOID* <br> *The purple blocks represent the forward propagation, and the red blocks represent the backward propagation.*  </center></caption>

<!-- 
For those of you who are expert in calculus (you don't need to be to do this assignment), the chain rule of calculus can be used to derive the derivative of the loss $\mathcal{L}$ with respect to $z^{[1]}$ in a 2-layer network as follows:

$$\frac{d \mathcal{L}(a^{[2]},y)}{{dz^{[1]}}} = \frac{d\mathcal{L}(a^{[2]},y)}{{da^{[2]}}}\frac{{da^{[2]}}}{{dz^{[2]}}}\frac{{dz^{[2]}}}{{da^{[1]}}}\frac{{da^{[1]}}}{{dz^{[1]}}} \tag{8} $$

In order to calculate the gradient $dW^{[1]} = \frac{\partial L}{\partial W^{[1]}}$, you use the previous chain rule and you do $dW^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial W^{[1]}}$. During the backpropagation, at each step you multiply your current gradient by the gradient corresponding to the specific layer to get the gradient you wanted.

Equivalently, in order to calculate the gradient $db^{[1]} = \frac{\partial L}{\partial b^{[1]}}$, you use the previous chain rule and you do $db^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial b^{[1]}}$.

This is why we talk about **backpropagation**.
!-->

Outline:
- LINEAR backward
- LINEAR -> ACTIVATION backward where ACTIVATION computes the derivative of either the ReLU or sigmoid activation
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID backward (whole model)

### 6.1 - Linear backward

For layer $l$, the linear part is: $z^{[l]} = {w^{[l]}}^T a^{[l-1]} + b^{[l]}$ (followed by an activation).

From chain rule, if we have $dz^{[l]} = \frac{\partial \mathcal{L} }{\partial z^{[l]}}$, then, to get $(dw^{[l]}, db^{[l]} da^{[l-1]})$ we have:

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption><center> **Figure 4** </center></caption>

$$ dz^{[l]} = \frac{\partial \mathcal{L} }{\partial w^{[l]}} = \frac{1}{m} a^{[l-1]} {dz^{[l]}}^T \tag{3}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dz^{[l](i)}\tag{4}$$
$$ da^{[l-1]} = \frac{\partial \mathcal{L} }{\partial a^{[l-1]}} = w^{[l]} dz^{[l]} \tag{5}$$


In [8]:
# linear_backward
def linear_backward(dz, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dz -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (a_prev, w, b) coming from the forward propagation in the current layer

    Returns:
    da_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dw -- Gradient of the cost with respect to W (current layer l), same shape as w
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    
    a_prev, w, b = cache
    m = a_prev.shape[1]

    dw = (1/m) * np.dot(a_prev, dz.T)
    db = (1/m) * np.sum(dz, axis=1, keepdims=True)
    da_prev = np.dot(w, dz)
    
    assert (da_prev.shape == a_prev.shape)
    assert (dw.shape == w.shape)
    assert (db.shape == b.shape)
    
    return da_prev, dw, db

### 6.2 - Linear-Activation backward

Next, we will create a function that merges the two helper functions: **`linear_backward`** and the backward step for the activation **`linear_activation_backward`**. 

We use following two backward functions from utils:
- **`sigmoid_backward`**: Implements the backward propagation for SIGMOID unit. We call it as follows:

```python
dz = sigmoid_backward(da, activation_cache)
```

- **`relu_backward`**: Implements the backward propagation for RELU unit. We call it as follows:

```python
dz = relu_backward(da, activation_cache)
```

If $g(.)$ is the activation function, 
`sigmoid_backward` and `relu_backward` compute $$dz^{[l]} = da^{[l]} * g'(z^{[l]}) \tag{6}$$.  

In [9]:
# linear_activation_backward
def linear_activation_backward(da, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    da -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    da_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as a_prev
    dw -- Gradient of the cost with respect to w (current layer l), same shape as w
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dz = relu_backward(da, activation_cache)
        da_prev, dw, db = linear_backward(dz, linear_cache)
        
    elif activation == "sigmoid":
        dz = sigmoid_backward(da, activation_cache)
        da_prev, dw, db = linear_backward(dz, linear_cache)
    
    return da_prev, dw, db

### 6.3 - L-Model Backward 

Now we will implement the backward function for the whole network. Recall that when we implemented the `L_model_forward` function, at each iteration, you stored a cache which contains (a_prev,w,b, and z). In the back propagation module, we will use those variables to compute the gradients. Therefore, in the `L_model_backward` function, we will iterate through all the hidden layers backward, starting from layer $L$. On each step, we will use the cached values for layer $l+1$ to backpropagate through layer $l$.


<img src="images/mn_backward.png" style="width:450px;height:300px;">
<caption><center>  **Figure 5** : Backward pass  </center></caption>

** Initializing backpropagation**:
To backpropagate through this network, we know that the output is, 
$a^{[L]} = \sigma(z^{[L]})$. Our code thus needs to compute `daL` $= \frac{\partial \mathcal{L}}{\partial a^{[L]}}$ from this.


In [10]:
# L_model_backward
def L_model_backward(aL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    aL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["da" + str(l)] = ... 
             grads["dw" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = aL.shape[1]
    if Y.shape[1] > 1:
        Y = Y.T
   
    # Initializing the backpropagation
    daL = - ( Y/aL.T - (1-Y)/(1-aL.T) ).T
    
    # Lth layer (SIGMOID -> LINEAR) gradients
    current_cache = caches[L-1]
    grads["da" + str(L)], grads["dw" + str(L)], grads["db" + str(L)] = linear_activation_backward(daL, current_cache, "sigmoid")
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        current_cache = caches[l]
        da_prev_temp, dw_temp, db_temp = linear_activation_backward(grads["da" + str(l + 2)], current_cache, "relu")
        grads["da" + str(l + 1)] = da_prev_temp
        grads["dw" + str(l + 1)] = dw_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

### 6.4 - Update Parameters


In [11]:
# update_parameters
def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    for l in range(L):
        parameters["w" + str(l+1)] = parameters["w" + str(l+1)] - learning_rate * grads["dw" + str(l + 1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l + 1)]
        
    return parameters


## 7 - What we'll do next in 'Deep Neural Network - Application.ipynb'

We will:
- export all the above helper functions to 'dnn_app_utils.py' 
- import 'dnn_app_utils.py' into 'Deep Neural Network - Application.ipynb'
- implement a two-layer neural network on 'catvnoncat' data
- implement an L-layer neural network on 'catvnoncat' data!